# Human Pose Estimation with YOLOv11

This notebook demonstrates how to use YOLOv11 for human detection and pose estimation.

**Requirements:**
```bash
pip install ultralytics opencv-python numpy
```

In [1]:
!pip install ultralytics opencv-python numpy

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
import os

In [3]:
# Load the YOLOv11 detection and pose models
det_model = YOLO('yolo11x.pt')
pose_model = YOLO('yolo11x-pose.pt')

100%|██████████| 109M/109M [00:02<00:00, 54.7MB/s] 


100%|██████████| 113M/113M [00:02<00:00, 44.5MB/s] 


In [8]:
# Define input and output paths
image_path = 'test_2.jpg'
save_path = 'test_2-pose_with_skeleton.jpg'

In [9]:

def draw_pose(image, keypoints_xy, keypoints_conf, thickness=2):
    if keypoints_xy is None or len(keypoints_xy) == 0 or keypoints_conf is None:
        return image

    skeleton = [
        (0, 1), (0, 2), (1, 3), (2, 4),
        (5, 6), (5, 7), (7, 9), (6, 8),
        (8, 10), (5, 11), (6, 12), (11, 12),
        (11, 13), (13, 15), (12, 14), (14, 16)
    ]

    for person_idx, (kpts, confs) in enumerate(zip(keypoints_xy, keypoints_conf)):
        for i, (x, y) in enumerate(kpts):
            if confs[i] > 0.3:
                cv2.circle(image, (int(x), int(y)), 3, (0, 255, 0), -1)

        for i, j in skeleton:
            if confs[i] > 0.3 and confs[j] > 0.3:
                pt1 = (int(kpts[i][0]), int(kpts[i][1]))
                pt2 = (int(kpts[j][0]), int(kpts[j][1]))
                cv2.line(image, pt1, pt2, (255, 0, 0), thickness)

    return image

In [10]:

image = cv2.imread(image_path)
if image is None:
    raise FileNotFoundError(f"Image not found at path: {image_path}")

image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

det_results = det_model(image_rgb)
bboxes = []

for result in det_results:
    for box in result.boxes:
        if int(box.cls[0]) == 0:
            bboxes.append(box.xyxy[0].cpu().numpy())

keypoints_xy = []
keypoints_conf = []

for bbox in bboxes:
    x1, y1, x2, y2 = map(int, bbox)
    person_crop = image_rgb[y1:y2, x1:x2]
    pose_results = pose_model(person_crop)

    for pose in pose_results:
        if pose.keypoints is not None:
            kpts = pose.keypoints.xy[0].cpu().numpy()
            confs = pose.keypoints.conf[0].cpu().numpy()
            kpts[:, 0] += x1
            kpts[:, 1] += y1
            keypoints_xy.append(kpts)
            keypoints_conf.append(confs)


0: 384x640 1 person, 3 bottles, 1 cup, 1 chair, 2 couchs, 1 dining table, 1 tv, 1 microwave, 27.1ms
Speed: 2.0ms preprocess, 27.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 480x640 1 person, 91.4ms
Speed: 2.1ms preprocess, 91.4ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)


In [11]:

annotated_image = draw_pose(image.copy(), keypoints_xy, keypoints_conf)
cv2.imwrite(save_path, annotated_image)


True

## Live